In [1]:
import math
import datetime
import string
import re
from glob import glob

import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
import yahoo_fin.stock_info as stock_info

from utils import *

In [3]:
all_tickers = get_all_usa_tickers()

In [13]:
ticker = "AAT"
yf_ticker = yf.Ticker(ticker)

In [14]:
yf_ticker.earnings_estimate

,avg,low,high,numberOfAnalysts
period,,,,
0q,0.0,0.0,0.0,0
+1q,0.0,0.0,0.0,0
0y,0.0,0.0,0.0,0
+1y,0.0,0.0,0.0,0


In [ ]:
parquet_paths = glob("DB/**/info.parquet")
for parquet_path in parquet_paths:
    df = pd.read_parquet(parquet_path)
    # df.columns = [camel_to_title_case(col) for col in df.columns]
    # df.to_parquet(parquet_path)